In [1]:
from tkinter import *
import socket
import ipaddress
from tkinter import messagebox

In [2]:
class ConnectForm:
    def on_closing(self):
        root.deiconify()
        self.master.destroy()
        global exitflag
        exitflag=1
    def GetHost(self):
        global HOST
        HOST=self.entry.get()
        try:
            socket.inet_pton(socket.AF_INET,HOST)
        except socket.error:
            messagebox.showerror("IP khong hop le", "Xin nhap lai")
            self.entry.delete(0, 'end')
        else:
            root.destroy()
    def __init__(self, master):
        self.master = master
        master.title("Connect")
        
        self.label = Label(master, text="Please connect to a server:")
        self.label.pack()

        self.entry = Entry(master)
        self.entry.pack()

        self.connect_button = Button(master, text="Connect",command=self.GetHost)
        self.connect_button.pack()
    

In [3]:
class RegisterForm:
    def on_closing(self):
        root.deiconify()
        self.master.destroy()
        global exitflag
        exitflag=1
    def Register_click(self):
        global ACT
        global PSW
        ACT=self.account_entry.get()
        PSW=self.password_entry.get()
        if (PSW==self.repassword_entry.get()):
            s.sendall(bytes(f"REQUEST REGISTER {ACT} {PSW}", encoding='utf-8'))
            data = s.recv(2048)
            data = data.decode('utf-8')
            print(data)
            data=data.split()
            if (data[0]=="DENIED" and data[2]=="EXITED"):
                messagebox.showerror("Tai khoan da ton tai","Vui long chon tai khoan khac" )
            if (data[0]=="ACCEPTED"):
                messagebox.showinfo("Dang ky thanh cong","Chuan bi login bang tai khoan dang ky")
                root.deiconify()
                self.master.destroy()
        else:
            messagebox.showerror("Xin nhap lai","Mat khau khong trung khop" )
            
        
    def __init__(self,master):
        self.master=master
        master.title("Register")
        def on_click_account(event):
            self.account_entry.configure(state=NORMAL)
            self.account_entry.delete(0, END)
            self.account_entry.unbind('<Button-1>', on_click_id_account)
        def on_click_password(event):
            self.password_entry.configure(state=NORMAL)
            self.password_entry.delete(0, END)
            self.password_entry.unbind('<Button-1>', on_click_id_password)
            self.password_entry.config(show="*")
        def on_click_repassword(event):
            self.repassword_entry.configure(state=NORMAL)
            self.repassword_entry.delete(0, END)
            self.repassword_entry.unbind('<Button-1>', on_click_id_repassword)
            self.repassword_entry.config(show="*")
            
        self.account_entry = Entry(master)
        self.account_entry.pack()
        self.account_entry.insert(0, "Account")
        #self.account_entry.configure(state=DISABLED)
        on_click_id_account = self.account_entry.bind('<Button-1>', on_click_account)
        
        self.password_entry = Entry(master)
        self.password_entry.pack()
        self.password_entry.insert(0, "Password")
        #self.password_entry.configure(state=DISABLED)
        on_click_id_password = self.password_entry.bind('<Button-1>', on_click_password)
        
        self.repassword_entry = Entry(master)
        self.repassword_entry.pack()
        self.repassword_entry.insert(0, "rePassword")
        #self.password_entry.configure(state=DISABLED)
        on_click_id_repassword = self.repassword_entry.bind('<Button-1>', on_click_repassword)

        self.register_button = Button(master, text="Register",width=10,command=self.Register_click)
        self.register_button.pack()

In [4]:
class LoginForm:
    def on_closing(self):
        root.deiconify()
        self.master.destroy()
        global exitflag
        exitflag=1
    def Login_click(self):
        global PSW,ACT
        PSW=self.password_entry.get()
        ACT=self.account_entry.get()
        root.destroy()
    def Register_click(self):
        global PSW,ACT
        root.withdraw()
        sub= Tk()
        RF=RegisterForm(sub)
        sub.protocol("WM_DELETE_WINDOW", RF.on_closing)
        sub.mainloop()
        
        
    def __init__(self,master):
        def on_click_account(event):
            self.account_entry.configure(state=NORMAL)
            self.account_entry.delete(0, END)
            self.account_entry.unbind('<Button-1>', on_click_id_account)
        def on_click_password(event):
            self.password_entry.configure(state=NORMAL)
            self.password_entry.delete(0, END)
            self.password_entry.unbind('<Button-1>', on_click_id_password)
            self.password_entry.config(show="*")
            
        self.master = master
        master.title("Login")
        
        self.label = Label(master, text="Please login the account:")
        self.label.pack()

        self.account_entry = Entry(master)
        self.account_entry.pack()
        self.account_entry.insert(0, "Account")
        #self.account_entry.configure(state=DISABLED)
        on_click_id_account = self.account_entry.bind('<Button-1>', on_click_account)
        
        self.password_entry = Entry(master)
        self.password_entry.pack()
        self.password_entry.insert(0, "Password")
        #self.password_entry.configure(state=DISABLED)
        on_click_id_password = self.password_entry.bind('<Button-1>', on_click_password)

        self.login_button = Button(master, text="Login",width=10,command=self.Login_click)
        self.login_button.pack()
        
        self.register_button = Button(master, text="Register",width=10,command=self.Register_click)
        self.register_button.pack()
    

In [5]:
PORT = 20044
HOST = ""
ACT=""
PSW=""
exitflag=0
root = Tk()
my_gui = ConnectForm(root)
root.protocol("WM_DELETE_WINDOW", my_gui.on_closing)
root.mainloop()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    data = s.recv(1024)
    print(data)
    s.sendall(b'NEW CLIENT CONNECTED')
    data = s.recv(1024)
    print(data)
    while exitflag==0:
        root = Tk()
        my_gui = LoginForm(root)
        root.protocol("WM_DELETE_WINDOW", my_gui.on_closing)
        root.mainloop()
        if exitflag==1:
            break
        print(f"REQUEST LOGIN {ACT} {PSW}")
        s.sendall(bytes(f"REQUEST LOGIN {ACT} {PSW}", encoding='utf-8'))
        data = s.recv(1024)
        print(data)
        data=data.decode('utf-8').split()
        if (data[0]=='DENIED'):
            continue
        break

b'ACCEPTED CONNECT'
b'HELLO CLIENT 1'
REQUEST LOGIN nxv 1
b'DENIED LOGIN'
ACCEPTED REGISTER
REQUEST LOGIN nxv 1
b'ACCEPTED LOGIN'
